# hydro_opendata.s3api.minio模块
用于读取minio中的数据

## 读取era5

In [1]:
from hydro_opendata.s3api.minio import ERA5LReader
import numpy as np
import geopandas as gpd

In [2]:
era5 = ERA5LReader()

In [3]:
start_time=np.datetime64("2021-06-01T00:00:00.000000000")
end_time=np.datetime64("2021-06-30T23:00:00.000000000")

- 通过指定四至范围读取

In [4]:
bbox=(121,39,123,40)
e1 = era5.open_dataset(data_variables=['Total precipitation'], start_time=start_time, end_time=end_time, dataset='wis', bbox=bbox)
e1

/home/zhujianfeng/.conda/envs/dev-env/lib/python3.10/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:  (time: 720, lon: 21, lat: 11)
Coordinates:
  * lat      (lat) float32 39.0 39.1 39.2 39.3 39.4 ... 39.6 39.7 39.8 39.9 40.0
  * lon      (lon) float32 121.0 121.1 121.2 121.3 ... 122.7 122.8 122.9 123.0
  * time     (time) datetime64[ns] 2021-06-01 ... 2021-06-30T23:00:00
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(24, 21, 11), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

- 通过矢量数据文件读取

In [5]:
shp = 'basin.geojson'
e2 = era5.from_shp(data_variables=['Total precipitation'], start_time=start_time, end_time=end_time, dataset='wis', shp=shp)
e2

<xarray.Dataset>
Dimensions:  (time: 720, lon: 6, lat: 5)
Coordinates:
  * lat      (lat) float32 39.8 39.9 40.0 40.1 40.2
  * lon      (lon) float32 122.3 122.4 122.5 122.6 122.7 122.8
  * time     (time) datetime64[ns] 2021-06-01 ... 2021-06-30T23:00:00
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(24, 6, 5), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

- 通过已有aoi对象读取

In [6]:
aoi = gpd.read_file(shp)
e3 = era5.from_aoi(data_variables=['Total precipitation'], start_time=start_time, end_time=end_time, dataset='wis', aoi=aoi)
e3

<xarray.Dataset>
Dimensions:  (time: 720, lon: 6, lat: 5)
Coordinates:
  * lat      (lat) float32 39.8 39.9 40.0 40.1 40.2
  * lon      (lon) float32 122.3 122.4 122.5 122.6 122.7 122.8
  * time     (time) datetime64[ns] 2021-06-01 ... 2021-06-30T23:00:00
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(24, 6, 5), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## 读取gpm

In [7]:
from hydro_opendata.s3api.minio import GPMReader

In [8]:
gpm = GPMReader()

In [9]:
start_time=np.datetime64("2023-06-01T00:00:00.000000000")
end_time=np.datetime64("2023-06-30T23:30:00.000000000")

- 通过指定四至范围读取

In [10]:
bbox=(-123,39,-121,40)
g1 = gpm.open_dataset(start_time=start_time, end_time=end_time, dataset='camels', bbox=bbox, time_resolution='30m')
g1

<xarray.DataArray 'precipitationCal' (time: 1440, lon: 20, lat: 10)>
dask.array<getitem, shape=(1440, 20, 10), dtype=float32, chunksize=(48, 20, 10), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 39.05 39.15 39.25 39.35 ... 39.65 39.75 39.85 39.95
  * lon      (lon) float32 -122.9 -122.8 -122.8 -122.7 ... -121.2 -121.2 -121.1
  * time     (time) datetime64[ns] 2023-06-01 ... 2023-06-30T23:30:00
Attributes:
    CodeMissingValue:  -9999.9
    DimensionNames:    time,lon,lat
    Units:             mm/hr
    units:             mm/hr

## 读取gfs

In [11]:
from hydro_opendata.s3api.minio import GFSReader

In [12]:
gfs = GFSReader()    

In [13]:
gfs.set_default_variable('tp')    # tp为降雨

- 通过指定四至范围读取

In [14]:
bbox=(121,39,123,40)
creation_date=np.datetime64("2023-06-01")
f1 = gfs.open_dataset(creation_date=creation_date, creation_time='00', dataset='wis', bbox=bbox)
f1

<xarray.Dataset>
Dimensions:     (lat: 5, lon: 9, step: 120, time: 1, valid_time: 120)
Coordinates:
  * lat         (lat) float64 39.0 39.25 39.5 39.75 40.0
  * lon         (lon) float64 121.0 121.2 121.5 121.8 ... 122.5 122.8 123.0
  * step        (step) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * time        (time) datetime64[ns] 2023-06-01
  * valid_time  (valid_time) datetime64[ns] 2023-06-01T01:00:00 ... 2023-06-06
Data variables:
    tp          (valid_time, step, lat, lon) float64 dask.array<chunksize=(24, 1, 5, 9), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0

- 通过矢量数据文件读取

In [15]:
shp = 'basin.geojson'
f2 = gfs.from_shp(creation_date=creation_date, creation_time='00', dataset='wis', shp=shp)
f2

<xarray.Dataset>
Dimensions:     (lat: 3, lon: 3, step: 120, time: 1, valid_time: 120)
Coordinates:
  * lat         (lat) float64 39.75 40.0 40.25
  * lon         (lon) float64 122.2 122.5 122.8
  * step        (step) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * time        (time) datetime64[ns] 2023-06-01
  * valid_time  (valid_time) datetime64[ns] 2023-06-01T01:00:00 ... 2023-06-06
Data variables:
    tp          (valid_time, step, lat, lon) float64 dask.array<chunksize=(24, 1, 3, 3), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0

- 通过已有aoi对象读取

In [16]:
aoi = gpd.read_file(shp)
f3 = gfs.from_aoi(creation_date=creation_date, creation_time='00', dataset='wis', aoi=aoi)
f3

<xarray.Dataset>
Dimensions:     (lat: 3, lon: 3, step: 120, time: 1, valid_time: 120)
Coordinates:
  * lat         (lat) float64 39.75 40.0 40.25
  * lon         (lon) float64 122.2 122.5 122.8
  * step        (step) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * time        (time) datetime64[ns] 2023-06-01
  * valid_time  (valid_time) datetime64[ns] 2023-06-01T01:00:00 ... 2023-06-06
Data variables:
    tp          (valid_time, step, lat, lon) float64 dask.array<chunksize=(24, 1, 3, 3), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0